# Using an API

In this example we will be using the `api.football-data.org` api.
The documentation for the Api can be found here https://www.football-data.org/ 


## Before starting make sure that you have read the API documentation

In [ ]:
# These are the libraries we will need to use

import requests
import json
import csv
import time
import pandas as pd

## A simple call

## We can use the JSON.dumps method to see the returned data or

## We can write it to a file and use other tools to look at it. 


In [ ]:
r=requests.get("http://api.football-data.org/v2/competitions/2021/matches", 
               params={'matchday': '1', 'season': '2018'},               
               headers={'X-Auth-Token':'ae3c0b8b3a544082889c24df68c7e951'})


# making the json more readable
#print(json.dumps(r.json(), indent=4))

# writing the json to a file
with open("games.json", "w") as outfile : outfile.write(json.dumps(r.json(), indent=4))


In [ ]:
# What it really looks like
print(r.text)  

## Using the same approach as we did for multiple files 
## We can make multiple calls by iterating through the matchdays

## Why might this code cause a problem?   - Read the API documents! 

In [ ]:
url = 'https://api.football-data.org/v2/competitions/2021/matches'
headers = { 'X-Auth-Token': '<your API token goes here>' }   # Your API Key

filters = {'matchday': '1', 'season': '2018'}

for md in range(1,39) :
    filters['matchday'] = str(md)
    r = requests.get(url, headers=headers, params=filters)
    print(r.url)
    


## Now we need to parse the json to extract the pieces of information we are going to need.

## First however, we will have alook at the saved JSON file

## We will start by extracting indivdual elements from the first match 
## (In Python this will be the index value of `0`).

In [ ]:
response = r.json()

print(response['matches'][0])

## Notice that this is a complete dictionary object. From which we can extract the values from the keys we want

## We need the Home team and the Away team and the Goals scored by each at Full Time.

## There is a lot more details in the dictionary object, so for fun we will also extract the score at Half Time. 

In [ ]:
print(response['matches'][0]['score'])

In [ ]:
print(response['matches'][0]['score']['fullTime']['homeTeam'])
print(response['matches'][0]['score']['fullTime']['awayTeam'])

In [ ]:
print(response['matches'][0]['homeTeam']['name'])
print(response['matches'][0]['awayTeam']['name'])

In [ ]:
print(response['matches'][0]['score']['halfTime']['homeTeam'])
print(response['matches'][0]['score']['halfTime']['awayTeam'])

For clarity we will assign these values to simple variables

In [ ]:
HomeTeam = response['matches'][0]['homeTeam']['name']
AwayTeam = response['matches'][0]['awayTeam']['name']
HomeTeamHalfTime = response['matches'][0]['score']['halfTime']['homeTeam']
AwayTeamHalfTime = response['matches'][0]['score']['halfTime']['awayTeam']
HomeTeamFullTime = response['matches'][0]['score']['fullTime']['homeTeam']
AwayTeamFullTime = response['matches'][0]['score']['fullTime']['awayTeam']

## As we want to save all of the results we will write them all to a simple `csv` file for future use.

## Because eventually we are going to retrieve the data for all of the `matchday`s. It might be useful to write this to the file as well.

## If we go back and look at the JSON, we can see that `matchday` is recorded as part of the `filter` entry

![Matchday](./images/matchday.jpg)

In [ ]:
Matchday = response['filters']['matchday']

## To write a csv file we will use the Python `csv` package.

## We will try out our code by just writing out the single row we have so far.
## In addition to the data elements, as it is a csv file we will also write out a header record with all of the column name in it

In [ ]:

fw =  open('results.csv', 'w') 
outfile = csv.writer(fw, delimiter=',', lineterminator='\r')

# The header record
outfile.writerow(["matchday", "Home_team", "Home_goals", "Away_team", "Away_goals", "HT_Home_goals", "HT_Away_Goals"])

#The data
outfile.writerow([Matchday, HomeTeam, AwayTeam, HomeTeamFullTime, AwayTeamFullTime, HomeTeamHalfTime, AwayTeamHalfTime] )
                      
# close the file    
fw.close()



We can see what this looks like by Browsing the file 

![Result](./images/results.jpg)


Now we are ready to start putting it all together.

1. Each API call gets us all of the results for a given Matchday.
2. We need to make a seperate API call for each of the 38 matchdays

In [ ]:
fw =  open('results.csv', 'w') 
outfile = csv.writer(fw, delimiter=',', lineterminator='\r')

# The header record
outfile.writerow(["matchday", "Home_team", "Away_team", "Home_goals", "Away_goals", "HT_Home_goals", "HT_Away_Goals"])

url = 'https://api.football-data.org/v2/competitions/2021/matches'
headers = { 'X-Auth-Token': 'ae3c0b8b3a544082889c24df68c7e951' }   # This is my API Key

filters = {'matchday': '1', 'season': '2018'}

for md in range(1,39) :
    filters['matchday'] = str(md)
    r = requests.get(url, headers=headers, params=filters)
    response = r.json()
    
    time.sleep(10)
    
    Matchday = response['filters']['matchday']
    for game in range(0,10) :
        HomeTeam = response['matches'][game]['homeTeam']['name']
        AwayTeam = response['matches'][game]['awayTeam']['name']
        HomeTeamHalfTime = response['matches'][game]['score']['halfTime']['homeTeam']
        AwayTeamHalfTime = response['matches'][game]['score']['halfTime']['awayTeam']
        HomeTeamFullTime = response['matches'][game]['score']['fullTime']['homeTeam']
        AwayTeamFullTime = response['matches'][game]['score']['fullTime']['awayTeam']
        
        #The data
        outfile.writerow([Matchday, HomeTeam, AwayTeam, HomeTeamFullTime, AwayTeamFullTime, HomeTeamHalfTime, AwayTeamHalfTime] )    
        
# close the file    
fw.close()

## With the results file we can generate a league table

In [ ]:

table = {}

f = open('results.csv', 'r')
f_csv = csv.reader(f, delimiter=',')
next(f_csv)

for line in f_csv:
    
    #print (line[0], line[1], line[2], line[3], line[4])
    if line[1] in table :                                    # repeat all of this for line[2] - Away team
        table[line[1]]['played'] += 1
        table[line[2]]['played'] += 1
        if line[3] > line[4] :
            table[line[1]]['won'] += 1
            table[line[1]]['points']  += 3
            table[line[2]]['lost'] += 1
        if line[3]  == line[4] :
            table[line[1]]['drawn'] += 1
            table[line[1]]['points']  += 1
            table[line[2]]['drawn'] += 1
            table[line[2]]['points']  += 1
        if line[3]  < line[4] :
            table[line[1]]['lost'] += 1
            table[line[2]]['won'] += 1
            table[line[2]]['points']  += 3
        table[line[1]]['GF'] += int(line[3]) 
        table[line[1]]['GA'] += int(line[4])
        table[line[2]]['GF'] += int(line[4]) 
        table[line[2]]['GA'] += int(line[3])
                    
    else :
        table[line[1]] = {}
        table[line[1]]['played'] = 0
        table[line[1]]['won'] = 0
        table[line[1]]['drawn'] = 0
        table[line[1]]['lost'] = 0
        table[line[1]]['GF'] = 0
        table[line[1]]['GA'] = 0
        table[line[1]]['points'] = 0
        table[line[1]]['played'] += 1
        table[line[2]] = {}
        table[line[2]]['played'] = 0
        table[line[2]]['won'] = 0
        table[line[2]]['drawn'] = 0
        table[line[2]]['lost'] = 0
        table[line[2]]['GF'] = 0
        table[line[2]]['GA'] = 0
        table[line[2]]['points'] = 0
        table[line[2]]['played'] += 1  
        if line[3] > line[4] :
            table[line[1]]['won'] += 1
            table[line[1]]['points']  += 3
            table[line[2]]['lost']  += 1            
        if line[3] == line[4] :
            table[line[1]]['drawn'] += 1
            table[line[1]]['points']  += 1
            table[line[2]]['drawn'] += 1
            table[line[2]]['points']  += 1
        if line[3]  < line[4] :
            table[line[1]]['lost'] += 1
            table[line[2]]['won'] += 1
            table[line[2]]['points']  += 3
        table[line[1]]['GF'] += int(line[3]) 
        table[line[1]]['GA'] += int(line[4])
        table[line[2]]['GF'] += int(line[4]) 
        table[line[2]]['GA'] += int(line[3])
                      

#  then need to flatten the dictionary and print
        
f.close()

# create the dataframe
League_Table = pd.DataFrame(columns = ['Team', 'W', 'D', 'L', 'GF', 'GA', 'GD', 'Points'])

for team in table :
    print(team, table[team]['won'], table[team]['drawn'], table[team]['lost'], table[team]['GF'], table[team]['GA'], (table[team]['GF'] - table[team]['GA']), table[team]['points'])
    League_Table = League_Table.append({'Team': team , 'W' : table[team]['won'], 'D' : table[team]['drawn'], 'L' : table[team]['lost'], 'GF' : table[team]['GF'], 'GA' : table[team]['GA'], 'GD' : (table[team]['GF'] - table[team]['GA']), 'Points' : table[team]['points']}, ignore_index=True)

print(League_Table.head())

# Sort the dataframe and save to a file
League_Table.sort_values(['Points', 'GD'], ascending=[False, False], inplace=True )
League_Table.to_csv('League_Table.csv', index = False)

## To get a half-time table I just need to change the comparisons (3 -> 5 and 4 -> 6)

In [ ]:
table = {}

f = open('results.csv', 'r')
f_csv = csv.reader(f, delimiter=',')
next(f_csv)

for line in f_csv:
    
    #print (line[0], line[1], line[2], line[3], line[4])
    if line[1] in table :                                    # repeat all of this for line[2] - Away team
        table[line[1]]['played'] += 1
        table[line[2]]['played'] += 1
        if line[5] > line[6] :
            table[line[1]]['won'] += 1
            table[line[1]]['points']  += 3
            table[line[2]]['lost'] += 1
        if line[5]  == line[6] :
            table[line[1]]['drawn'] += 1
            table[line[1]]['points']  += 1
            table[line[2]]['drawn'] += 1
            table[line[2]]['points']  += 1
        if line[5]  < line[6] :
            table[line[1]]['lost'] += 1
            table[line[2]]['won'] += 1
            table[line[2]]['points']  += 3
        table[line[1]]['GF'] += int(line[3]) 
        table[line[1]]['GA'] += int(line[4])
        table[line[2]]['GF'] += int(line[4]) 
        table[line[2]]['GA'] += int(line[3])
                    
    else :
        table[line[1]] = {}
        table[line[1]]['played'] = 0
        table[line[1]]['won'] = 0
        table[line[1]]['drawn'] = 0
        table[line[1]]['lost'] = 0
        table[line[1]]['GF'] = 0
        table[line[1]]['GA'] = 0
        table[line[1]]['points'] = 0
        table[line[1]]['played'] += 1
        table[line[2]] = {}
        table[line[2]]['played'] = 0
        table[line[2]]['won'] = 0
        table[line[2]]['drawn'] = 0
        table[line[2]]['lost'] = 0
        table[line[2]]['GF'] = 0
        table[line[2]]['GA'] = 0
        table[line[2]]['points'] = 0
        table[line[2]]['played'] += 1  
        if line[5] > line[6] :
            table[line[1]]['won'] += 1
            table[line[1]]['points']  += 3
            table[line[2]]['lost']  += 1            
        if line[5] == line[6] :
            table[line[1]]['drawn'] += 1
            table[line[1]]['points']  += 1
            table[line[2]]['drawn'] += 1
            table[line[2]]['points']  += 1
        if line[5]  < line[6] :
            table[line[1]]['lost'] += 1
            table[line[2]]['won'] += 1
            table[line[2]]['points']  += 3
        table[line[1]]['GF'] += int(line[3]) 
        table[line[1]]['GA'] += int(line[4])
        table[line[2]]['GF'] += int(line[4]) 
        table[line[2]]['GA'] += int(line[3])
                      

#  then need to flatten the dictionary and print
        
f.close()

# create the dataframe
League_Table = pd.DataFrame(columns = ['Team', 'W', 'D', 'L', 'GF', 'GA', 'GD', 'Points'])

for team in table :
    #print(team, table[team]['won'], table[team]['drawn'], table[team]['lost'], table[team]['GF'], table[team]['GA'], (table[team]['GF'] - table[team]['GA']), table[team]['points'])
    League_Table = League_Table.append({'Team': team , 'W' : table[team]['won'], 'D' : table[team]['drawn'], 'L' : table[team]['lost'], 'GF' : table[team]['GF'], 'GA' : table[team]['GA'], 'GD' : (table[team]['GF'] - table[team]['GA']), 'Points' : table[team]['points']}, ignore_index=True)

#print(League_Table.head())

# Sort the dataframe and save to a file
League_Table.sort_values(['Points', 'GD'], ascending=[False, False], inplace=True )
League_Table.to_csv('League_Table (HT).csv', index = False)
